In [1]:
pip install transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 26.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 12.0 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

MODEL_PATH = "/kaggle/input/llama-3.2/transformers/3b-instruct/1"
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, trust_remote_code=True)
# tokenizer.padding_side = "right"
# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.add_eos_token = True

print(tokenizer.bos_token, tokenizer.eos_token)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype=torch.float16, # Use float16 for memory efficiency on GPUs
    device_map="auto" # Automatically maps model to available devices (like GPU)
)

# Set pad_token_id to eos_token_id for open-end generation
tokenizer.pad_token_id = tokenizer.eos_token_id

# Create a text-generation pipeline
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto"
)

2025-07-23 13:56:29.655148: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753278989.849927      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753278989.903990      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


<|begin_of_text|> <|eot_id|>


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [14]:
import pandas as pd 

df = pd.read_csv("/kaggle/input/nlp-pw/aug_data.csv")
df.Text[0]

'Che, quindi, il metodo di valutazione è, o no, fondato è Quaestio fatti (ET in Iuris) che, alle prove archeologiche, riguarda tutto il tempo della verifica giudiziaria (e, pertanto, alla valutazione riservata a questo seggio giurisdizionale, in aggiunta al contraddittorio delle parti) e che rimane estranea alla definizione del contenuto necessario (Act. Commertoing)'

In [25]:
# Example for an instruction-tuned model
system_message = "Sei un assistente che prende in input delle frasi estratte da documenti legali VAT ITA e genera nuove frasi simili. Non sono domande, cerca di creare nuove frasi con lo stesso senso. Tono formale e legale. Questo è un task di data augmentation. Dammi una sola risposta."
user_message = df.Text[10]

prompt = f"""<|start_header_id|>system<|end_header_id|>
{system_message}<|eot_id|>
<|start_header_id|>user<|end_header_id|>
{user_message}<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""

sequences = generator(
    prompt,
    do_sample=True,
    top_k=10,
    num_return_sequences=3,
    temperature = 0.8,
    eos_token_id=tokenizer.eos_token_id,
    max_length=500, # Adjust max_length as needed
)

generated_text = sequences[0]['generated_text']
# Extract only the assistant's response
assistant_response = generated_text[len(prompt):].strip()
print("Response:", assistant_response)

Both `max_new_tokens` (=256) and `max_length`(=500) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Response: La notificazione della sospensione del pagamento dei dividendi e la valutazione monetaria emessa a carico di una società di persone estinte sono azioni legittime, poiché la società, essendo estinta, non può più onorare i suoi obblighi di pagamento, trasferendo così i propri obblighi ai successori dei suoi debiti, inclusi quelli relativi alla previdenza sociale.


In [22]:
# Example for an instruction-tuned model
system_message = "Sei un assistente che deve svolgere un task di label classification. Il task consistente se il testo che ti passo è una premessa o conclusione. Risposta tra premessa oppure conclusione"
user_message = 'Quanto è grande il tuo context window'

prompt = f"""<|start_header_id|>system<|end_header_id|>
{system_message}<|eot_id|>
<|start_header_id|>user<|end_header_id|>
{user_message}<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""

sequences = generator(
    prompt,
    do_sample=True,
    top_k=10,
    num_return_sequences=3,
    eos_token_id=tokenizer.eos_token_id,
    max_length=500, # Adjust max_length as needed
)

generated_text = sequences[0]['generated_text']
# Extract only the assistant's response
assistant_response = generated_text[len(prompt):].strip()
print("Response:", assistant_response)

Both `max_new_tokens` (=256) and `max_length`(=500) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Response: Il mio "context window" è di 4096 token


In [10]:
# Put model in eval mode
model.eval()

# Your prompt
prompt = "che cosa è un large language model?"

# Tokenize input
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# Generate output
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=500,
        do_sample=True,
        temperature=0.7,
        top_p=0.9
    )

# Decode output
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


che cosa è un large language model? 
Un large language model è un modello di intelligenza artificiale (IA) che è stato sviluppato per processare e comprendere grandi quantità di testo. Questi modelli sono stati sviluppati utilizzando tecniche di apprendimento automatico e sono in grado di imparare e rappresentare un vasto spettro di conoscenze e concetti. 

I large language models sono stati utilizzati in una varietà di applicazioni, tra cui: 

- Ricerca scientifica: per aiutare gli scienziati a comprendere e analizzare grandi quantità di dati.
- Assistenza virtuale: per creare chatbot e altri sistemi di assistenza che possono comprendere e rispondere a domande e richieste dei utenti.
- Traduzione: per aiutare a tradurre testi da un'lingua all'altra.
- Scrittura automatica: per aiutare a generare testi automaticamente, come ad esempio articoli di notizie o descrizioni di prodotti. 

Alcuni esempi di large language models includono: 

- BERT (Bidirectional Encoder Representations from T